# Interactive Pareto Explorer

Use Plotly to explore a Pareto front interactively. Click a point to inspect the
decision variables and re-evaluate objectives/constraints.

Requirements:
- plotly
- ipywidgets

If missing, install with:

```bash
pip install plotly ipywidgets
```


In [ ]:
import numpy as np

HAS_PLOTLY = True
try:
    import plotly.graph_objects as go
    import ipywidgets as widgets
    from IPython.display import display
except ImportError as exc:
    HAS_PLOTLY = False
    print("Plotly and ipywidgets are required for the explorer.")
    print(f"Import error: {exc}")


In [ ]:
from vamos import optimize, OptimizeConfig, NSGAIIConfig
from vamos.foundation.problem.zdt1 import ZDT1Problem
from vamos.foundation.problem.zdt2 import ZDT2Problem
from vamos.foundation.problem.zdt3 import ZDT3Problem
from vamos.foundation.problem.dtlz import DTLZ2Problem
from vamos.foundation.problem.wfg import WFG4Problem

PROBLEMS = {
    "ZDT1 (2 obj)": lambda: ZDT1Problem(n_var=12),
    "ZDT2 (2 obj)": lambda: ZDT2Problem(n_var=12),
    "ZDT3 (2 obj)": lambda: ZDT3Problem(n_var=12),
    "DTLZ2 (3 obj)": lambda: DTLZ2Problem(n_var=12, n_obj=3),
}

if "WFG4 (3 obj)" in PROBLEMS:
    pass
else:
    PROBLEMS["WFG4 (3 obj)"] = lambda: WFG4Problem(n_var=10, n_obj=3)

if not HAS_PLOTLY:
    problem = None
    result = None
    print("Plotly and ipywidgets are required for the explorer.")
else:
    problem_dropdown = widgets.Dropdown(
        options=list(PROBLEMS.keys()),
        value="ZDT2 (2 obj)",
        description="Problem:",
        style={"description_width": "initial"},
    )
    eval_slider = widgets.IntSlider(
        value=8000,
        min=1000,
        max=20000,
        step=1000,
        description="Max evals:",
        style={"description_width": "initial"},
    )
    seed_slider = widgets.IntSlider(
        value=42,
        min=0,
        max=999,
        step=1,
        description="Seed:",
        style={"description_width": "initial"},
    )
    run_button = widgets.Button(description="Run", button_style="primary")
    status = widgets.Output()

    problem = None
    result = None

    def run_experiment(_=None):
        nonlocal problem, result
        with status:
            status.clear_output()
            label = problem_dropdown.value
            print(f"Running: {label} (max evals={eval_slider.value}, seed={seed_slider.value})")
        try:
            problem = PROBLEMS[label]()
            config = (
                NSGAIIConfig()
                .pop_size(56)
                .offspring_size(14)
                .crossover("blx_alpha", prob=0.88, alpha=0.94, repair="clip")
                .mutation("non_uniform", prob="0.45/n", perturbation=0.3)
                .selection("tournament", pressure=9)
                .survival("nsga2")
                .repair("round")
                .external_archive(size=56, archive_type="hypervolume")
                .engine("numpy")
                .fixed()
            )
            result = optimize(
                OptimizeConfig(
                    problem=problem,
                    algorithm="nsgaii",
                    algorithm_config=config,
                    termination=("n_eval", int(eval_slider.value)),
                    seed=int(seed_slider.value),
                )
            )
            with status:
                print("Run complete:", result)
        except Exception as exc:
            with status:
                print(f"Run failed: {exc}")

    run_button.on_click(run_experiment)
    display(widgets.VBox([widgets.HBox([problem_dropdown, eval_slider, seed_slider, run_button]), status]))

    run_experiment()




In [ ]:
from vamos import pareto_filter
from vamos.engine.algorithm.components.population import evaluate_population_with_constraints

if result is None or problem is None:
    F_all = None
    X_all = None
    front_mask = np.zeros((0,), dtype=bool)
else:
    F_all = result.F
    X_all = result.X

    def pareto_mask(F):
        front = pareto_filter(F)
        if front is None or front.size == 0:
            return np.zeros((0,), dtype=bool)
        mask = np.zeros(F.shape[0], dtype=bool)
        for i, row in enumerate(F):
            if np.any(np.all(np.isclose(front, row, rtol=1e-12, atol=1e-12), axis=1)):
                mask[i] = True
        return mask

    def format_vec(vec, max_items=8):
        arr = np.asarray(vec).ravel()
        head = np.array2string(arr[:max_items], precision=4, separator=", ")
        if arr.size > max_items:
            return f"{head} ... (+{arr.size - max_items} more)"
        return head

    front_mask = pareto_mask(F_all) if F_all is not None else np.zeros((0,), dtype=bool)


In [ ]:
if not HAS_PLOTLY:
    print("Plotly and ipywidgets are required for this cell.")
elif result is None or problem is None:
    print("No results available to visualize. Use the Run button first.")
elif F_all is None or X_all is None or F_all.size == 0:
    print("No solutions available to visualize.")
elif F_all.shape[1] not in (2, 3):
    print("This explorer currently supports 2 or 3 objectives.")
else:
    n_obj = F_all.shape[1]
    mode = widgets.ToggleButtons(
        options=[("Front only", "front"), ("All points", "all")],
        value="front",
        description="View:",
        button_style="",
    )
    info = widgets.Output()

    if n_obj == 2:
        trace = go.Scattergl(
            x=[],
            y=[],
            mode="markers",
            marker={"size": 8, "opacity": 0.8},
            customdata=[],
            hovertemplate="f1=%{x:.4f}<br>f2=%{y:.4f}<extra></extra>",
        )
        fig = go.FigureWidget(data=[trace])
        fig.update_layout(
            title="Pareto Explorer",
            xaxis_title="Objective 1",
            yaxis_title="Objective 2",
            height=520,
        )
    else:
        trace = go.Scatter3d(
            x=[],
            y=[],
            z=[],
            mode="markers",
            marker={"size": 4, "opacity": 0.8},
            customdata=[],
            hovertemplate="f1=%{x:.4f}<br>f2=%{y:.4f}<br>f3=%{z:.4f}<extra></extra>",
        )
        fig = go.FigureWidget(data=[trace])
        fig.update_layout(
            title="Pareto Explorer",
            scene={
                "xaxis_title": "Objective 1",
                "yaxis_title": "Objective 2",
                "zaxis_title": "Objective 3",
            },
            height=520,
        )

    state = {"indices": np.array([], dtype=int)}

    def update_plot(_=None):
        use_front = mode.value == "front"
        if use_front:
            mask = front_mask
        else:
            mask = np.ones(F_all.shape[0], dtype=bool)
        idx = np.flatnonzero(mask)
        state["indices"] = idx
        F_view = F_all[idx]
        fig.data[0].x = F_view[:, 0]
        fig.data[0].y = F_view[:, 1]
        if n_obj == 3:
            fig.data[0].z = F_view[:, 2]
        fig.data[0].customdata = idx
        fig.update_layout(title=f"Pareto Explorer ({'front' if use_front else 'all'})")

    def on_click(trace, points, _):
        if not points.point_inds:
            return
        local_idx = points.point_inds[0]
        global_idx = int(state["indices"][local_idx])
        x = X_all[global_idx]
        f = F_all[global_idx]
        F_eval, G_eval = evaluate_population_with_constraints(problem, x[None, :])

        with info:
            info.clear_output()
            print(f"Index: {global_idx}")
            print(f"Objectives (stored): {format_vec(f)}")
            print(f"Objectives (re-eval): {format_vec(F_eval[0])}")
            diff = F_eval[0] - f
            print(f"Delta (re-eval - stored): {format_vec(diff)}")
            if G_eval is not None:
                print(f"Constraints: {format_vec(G_eval[0])}")
            print(f"Decision vars: {format_vec(x)}")

    fig.data[0].on_click(on_click)
    mode.observe(update_plot, names="value")
    update_plot()

    display(widgets.VBox([mode, fig, info]))
